In [15]:
from IMPORTS import *

In [2]:
list1 = [1, 2, 3, 4, 5]  
  
print(random.sample(list1,5))

[4, 5, 1, 2, 3]


In [3]:
random.random()

0.5477507308232752

In [5]:
11 % 5 == 0

False

In [13]:
t = torch.tensor([[1,20],[3,40]])
torch.gather(t, 0, torch.tensor([[0,0],[1,0]]))

tensor([[ 1, 20],
        [ 3, 20]])

In [19]:
import collections
my_deq = collections.deque(maxlen=3)

In [147]:
my_deq.append(a)
a += 1
my_deq

deque([109, 110, 111])

In [36]:
a = 1